In [11]:
import tensorflow as tf
from tensorflow.python.summary.summary_iterator import summary_iterator

path_to = "../logs/sketch_transformer_experiment_0/1762736869.7878423/events.out.tfevents.1762736869.DESKTOP-6UC3P7J.28764.1"
def extract_dictionary():
    for e in tf.train.summary_iterator(path_to):
        for v in e.summary.value:
            print(v)

In [13]:
import os, glob
import tensorflow as tf
from tensorflow.python.framework import tensor_util  # for tensor values

path = r"../logs/sketch_transformer_experiment_0/1762736869.7878423/events.out.tfevents.1762736869.DESKTOP-6UC3P7J.28764.1"
# On Windows, make sure the path is correct; prefer an absolute "C:\\..." path.

def dump_events(path_or_dir):
    files = []
    if os.path.isdir(path_or_dir):
        files = glob.glob(os.path.join(path_or_dir, "events.out.tfevents.*"))
    elif os.path.isfile(path_or_dir):
        files = [path_or_dir]
    else:
        raise FileNotFoundError(f"Path not found: {path_or_dir}")

    for f in files:
        for e in tf.compat.v1.train.summary_iterator(f):
            for v in e.summary.value:
                # Print scalars and tensors nicely
                if v.HasField("simple_value"):
                    print(e.step, v.tag, v.simple_value)
                elif v.HasField("tensor"):
                    try:
                        val = tensor_util.MakeNdarray(v.tensor)
                        print(e.step, v.tag, val)
                    except Exception:
                        print(e.step, v.tag, "<non-scalar>")

dump_events(path)

In [17]:
from tensorboard.backend.event_processing import event_accumulator

# Path to one event file or a directory containing event files
event_path = "../logs"

ea = event_accumulator.EventAccumulator(event_path)
ea.Reload()  # Loads events from file

# List available tags (e.g. scalars, images, histograms)
print("Available scalar tags:", ea.Tags()["scalars"])

# Get all scalar data for a tag
#scalars = ea.Scalars("Loss/train")

# Each entry is a namedtuple: (wall_time, step, value)
##for s in scalars:
#    print(f"Step {s.step} -> {s.value}")

Available scalar tags: []


In [42]:
import tensorflow as tf

file_path = "../logs/sketch_transformer_experiment_0/events.out.tfevents.1762733017.DESKTOP-6UC3P7J.22876.8"


for e in tf.compat.v1.train.summary_iterator(file_path):
    print(f"Step: {e.step}, Wall time: {e.wall_time}")
    for v in e.summary.value:
        print(f"  Tag: {v.tag}, Value: {v.simple_value}")

Step: 0, Wall time: 1762733017.8865252
Step: 0, Wall time: 1762733020.9836898
Step: 0, Wall time: 1762733020.984215
Step: 0, Wall time: 1762733021.7816718
  Tag: Targets/ValTokens, Value: 0.0
Step: 0, Wall time: 1762733717.0144799
  Tag: Loss/Train, Value: 2.1321747303009033
Step: 0, Wall time: 1762733858.983716
  Tag: Predictions/ValTokens, Value: 0.0
Step: 0, Wall time: 1762733858.9842446
  Tag: Loss/Val, Value: 2.1101455688476562
Step: 0, Wall time: 1762733858.9847455
  Tag: Accuracy/ValNextToken, Value: 0.13740158081054688
Step: 0, Wall time: 1762733858.9895346
  Tag: Perplexity/Val, Value: 8.249442100524902


In [26]:
import glob
import os
import tensorflow as tf

# TF2 users: the iterator lives under compat.v1
summary_iterator = tf.compat.v1.train.summary_iterator

# Proto definitions for the HParams plugin
from tensorboard.plugins.hparams import api_pb2

def _val_to_python(v):  # tensorboard.hparams Value -> py value
    kind = v.WhichOneof("kind")
    if kind == "string_value":
        return v.string_value
    if kind == "bool_value":
        return v.bool_value
    if kind == "number_value":
        return v.number_value
    return None

def extract_hparams_from_event_file(event_path):
    runs = []  # list of dicts: {"run": event_path, "hparams": {...}}
    for e in summary_iterator(event_path):
        if not e.summary.value:
            continue
        for v in e.summary.value:
            md = v.metadata
            if not md or not md.plugin_data:
                continue
            pd = md.plugin_data
            # HParams plugin uses plugin_name == "hparams"
            if pd.plugin_name != "hparams":
                continue

            hp_data = api_pb2.HParamsPluginData()
            hp_data.ParseFromString(pd.content)
            which = hp_data.WhichOneof("data")  # "experiment" | "session_start_info" | "session_end_info"

            if which == "session_start_info":
                ss = hp_data.session_start_info
                hps = {k: _val_to_python(val) for k, val in ss.hparams.items()}
                runs.append({"run": event_path, "group_name": ss.group_name, "hparams": hps})
    return runs

def extract_hparams(logdir_or_file):
    paths = []
    if os.path.isdir(logdir_or_file):
        # You can add deeper globs if you have nested runs
        paths = glob.glob(os.path.join(logdir_or_file, "events.out.tfevents.*"))
    else:
        paths = [logdir_or_file]

    all_runs = []
    for p in paths:
        all_runs.extend(extract_hparams_from_event_file(p))
    return all_runs

# --- usage ---
logpath = "../logs/sketch_transformer_experiment_0/events.out.tfevents.1762733017.DESKTOP-6UC3P7J.22876.8"  # can be a directory or a single event file
for run in extract_hparams(logpath):
    print("RUN:", run["run"])
    if run.get("group_name"):
        print(" group:", run["group_name"])
    for k, v in run["hparams"].items():
        print(f"  {k}: {v}")
    print()

In [36]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import tensor_util  # safe to use; stable API is limited

summary_iterator = tf.compat.v1.train.summary_iterator

def try_decode_tensor_to_str(t):
    arr = tensor_util.MakeNdarray(t)
    # Common cases: scalar bytes, 1D array of bytes/strings
    if arr.dtype.type is np.object_:
        # e.g., array([b'...']), pick first non-empty
        for x in arr.flatten():
            if isinstance(x, (bytes, bytearray)):
                try:
                    return x.decode("utf-8", errors="ignore")
                except Exception:
                    pass
            elif isinstance(x, str):
                return x
        return None
    if arr.dtype.type is np.bytes_:
        try:
            return arr.tobytes().decode("utf-8", errors="ignore")
        except Exception:
            return None
    if arr.ndim == 0:
        return str(arr.item())
    return str(arr)

def scan_textlike_hparams(event_path):
    hits = []
    for e in summary_iterator(event_path):
        for v in e.summary.value:
            tag = v.tag.lower()
            if ("hparam" in tag) or ("hyperparam" in tag) or ("config" in tag):
                if v.HasField("tensor"):
                    s = try_decode_tensor_to_str(v.tensor)
                    if s:
                        # Try JSON parse, otherwise keep raw text
                        try:
                            hits.append((v.tag, json.loads(s)))
                        except Exception:
                            hits.append((v.tag, s))
                elif v.HasField("simple_value"):
                    hits.append((v.tag, v.simple_value))
                elif v.HasField("metadata") and v.metadata.display_name:
                    hits.append((v.tag, v.metadata.display_name))
    return hits

# --- usage ---
event_file = "../logs/sketch_transformer_experiment_0/1762733017.8895097/events.out.tfevents.1762733017.DESKTOP-6UC3P7J.22876.9"  # can be a directory or a single event file
for run in extract_hparams(logpath):
    print("RUN:", run["run"])
for tag, content in scan_textlike_hparams(event_file):
    print(f"[{tag}]")
    print(content)
    print()

In [28]:
from tensorboard.backend.event_processing import event_accumulator
from tensorboard.plugins.hparams.plugin_data_pb2 import HParamsPluginData

path_str = "../logs/sketch_transformer_experiment_0/events.out.tfevents.1762733017.DESKTOP-6UC3P7J.22876.8"
ea = event_accumulator.EventAccumulator(path_str, 
                                        size_guidance={'tensors': 0} 
                                        )
ea.Reload()

data = ea._plugin_to_tag_to_content["hparams"]["_hparams_/session_start_info"]
hparam_data = HParamsPluginData.FromString(data).session_start_info.hparams
hparam_dict = {key: hparam_data[key].ListFields()[0][1] for key in hparam_data.keys()}

KeyError: '_hparams_/session_start_info'

In [31]:
from tensorboard.backend.event_processing import directory_loader
from tensorboard.plugins.hparams import api_pb2

run_dir = r"../logs/sketch_transformer_experiment_0"
dl = directory_loader.DirectoryLoader(run_dir)
for ev_file in dl.Load():
    for e in ev_file.Load():
        if not getattr(e, "summary", None):
            continue
        for v in e.summary.value:
            md = getattr(v, "metadata", None)
            if not (md and md.plugin_data and md.plugin_data.plugin_name == "hparams"):
                continue
            hp = api_pb2.HParamsPluginData()
            hp.ParseFromString(md.plugin_data.content)
            if hp.WhichOneof("data") == "session_start_info":
                ss = hp.session_start_info
                def _to_py(val):
                    kind = val.WhichOneof("kind")
                    return getattr(val, kind) if kind else None
                hparams = {k: _to_py(v) for k, v in ss.hparams.items()}
                print(hparams)

TypeError: DirectoryLoader.__init__() missing 1 required positional argument: 'loader_factory'

In [32]:
import glob, tensorflow as tf, os
files = glob.glob(os.path.join(logdir, "events.out.tfevents.*"))
print("files:", files)
print("records per file:", [sum(1 for _ in tf.data.TFRecordDataset([f])) for f in files])

files: []
records per file: []


In [33]:
import tensorflow as tf
from tensorboard.plugins.hparams import plugin_data_pb2  # message types
from google.protobuf.json_format import MessageToDict

path = "../logs/sketch_transformer_experiment_0/events.out.tfevents.1762733017.DESKTOP-6UC3P7J.22876.8"

experiment = None
trials = []  # list of dicts: {"run": global_step or wall_time, "hparams": {...}}

for e in tf.compat.v1.train.summary_iterator(path):
    if not e.summary.value:
        continue
    for v in e.summary.value:
        md = v.metadata
        if md.plugin_data.plugin_name != "hparams":
            continue

        blob = md.plugin_data.content

        # Try as Experiment config
        exp = plugin_data_pb2.Experiment()
        try:
            exp.ParseFromString(blob)
            experiment = MessageToDict(exp, preserving_proto_field_name=True)
            continue
        except Exception:
            pass

        # Try as SessionStartInfo (actual hyperparameter values for a run)
        ssi = plugin_data_pb2.SessionStartInfo()
        try:
            ssi.ParseFromString(blob)
            ssi_dict = MessageToDict(ssi, preserving_proto_field_name=True)
            # Convert list of {name, {string_value/float_value/bool_value}} to a plain dict
            hp_dict = {}
            for hp in ssi.hparams:
                name = hp["name"]
                # pick whichever value field exists
                val = hp.get("string_value", hp.get("float_value", hp.get("bool_value")))
                hp_dict[name] = val
            trials.append({"step": e.step, "wall_time": e.wall_time, "hparams": hp_dict})
            continue
        except Exception:
            pass

        # Try as SessionEndInfo (optional; not always needed)
        sei = plugin_data_pb2.SessionEndInfo()
        try:
            sei.ParseFromString(blob)
            # You could read status here if you care.
        except Exception:
            pass

print("Experiment schema:", experiment)
print("Found", len(trials), "trial(s). Example:", trials[:1])

Experiment schema: None
Found 0 trial(s). Example: []


In [35]:
import json
from tensorflow.python.framework import tensor_util

path = "../logs/sketch_transformer_experiment_0/1762733017.8895097/events.out.tfevents.1762733017.DESKTOP-6UC3P7J.22876.9"
hparam_json_hits = []
scalar_hparams = {}

for e in tf.compat.v1.train.summary_iterator(path):
    for v in e.summary.value:
        tag = v.tag or ""
        # Text summary (JSON dumped as bytes in TensorProto)
        if tag.lower().startswith("hparams") and v.tensor.ByteSize() > 0:
            arr = tensor_util.MakeNdarray(v.tensor)
            if arr.size == 1 and isinstance(arr.item(), (bytes, str)):
                txt = arr.item().decode() if isinstance(arr.item(), bytes) else arr.item()
                try:
                    hparam_json_hits.append(json.loads(txt))
                except Exception:
                    pass

        # Scalar summaries masquerading as hparams
        if tag.startswith("hparams/") and v.HasField("simple_value"):
            name = tag.split("/", 1)[1]
            scalar_hparams[name] = v.simple_value

print("JSON-like hparams:", hparam_json_hits[:1])
print("Scalar-like hparams:", scalar_hparams)

JSON-like hparams: []
Scalar-like hparams: {}


In [38]:
import glob, tensorflow as tf

for path in glob.glob("/logs/sketch_transformer_experiment_0/**/events.out.tfevents.*", recursive=True):
    print("\nInspecting:", path)
    try:
        for e in tf.compat.v1.train.summary_iterator(path):
            for v in e.summary.value:
                print("  ", v.tag)
            break  # just print first event
    except Exception as ex:
        print("  [error reading]", ex)

In [39]:
import os
import tensorflow as tf

# use the v1 iterator in TF2+
summary_iterator = tf.compat.v1.train.summary_iterator

# protobufs for the HParams plugin
from tensorboard.plugins.hparams import api_pb2

def _hp_value_to_py(v):
    # api_pb2.HParamValue is a oneof: number_value / string_value / bool_value
    if v.HasField("number_value"):
        return v.number_value
    if v.HasField("string_value"):
        return v.string_value
    if v.HasField("bool_value"):
        return v.bool_value
    return None

def extract_hparams_from_event_file(event_path):
    """Yield dicts of hparams found in the given events file."""
    if not os.path.exists(event_path):
        raise FileNotFoundError(event_path)

    for e in summary_iterator(event_path):
        if not e.summary.value:
            continue

        for v in e.summary.value:
            md = v.metadata
            if not md.plugin_data.plugin_name:
                continue

            # HParams live under plugin_name == "hparams"
            if md.plugin_data.plugin_name == "hparams":
                pd = api_pb2.HParamsPluginData()
                pd.ParseFromString(md.plugin_data.content)

                # There are three oneof variants: experiment, session_start_info, session_end_info
                if pd.HasField("session_start_info"):
                    ssi = pd.session_start_info
                    hp = {k: _hp_value_to_py(v) for k, v in ssi.hparams.items()}
                    yield {
                        "wall_time": e.wall_time,
                        "step": e.step,
                        "group_name": ssi.group_name,      # may be empty
                        "session_name": ssi.session_name,  # may be empty
                        "hparams": hp,
                    }

                # You could also read experiment metadata or session_end_info if you want
                # elif pd.HasField("experiment"):
                #     ...

# ---- usage ----
if __name__ == "__main__":
    path_to = "../logs/sketch_transformer_experiment_0/events.out.tfevents.1762733017.DESKTOP-6UC3P7J.22876.8"  # your exact file
    for record in extract_hparams_from_event_file(path_to):
        print("Session start @ step", record["step"])
        for k, v in record["hparams"].items():
            print(f"  {k}: {v}")

FileNotFoundError: ../logs/sketch_transformer_experiment_0/events.out.tfevents.1762733017.DESKTOP-6UC3P7J.22876.8

In [79]:
import os
import tempfile
# Define some constants
N_RUNS = 3
N_EVENTS = 2
# Prepare temp dirs for storing event files
tmpdirs = {}

tmpdirs['torch'] = tempfile.TemporaryDirectory()
from torch.utils.tensorboard import SummaryWriter
log_dir = "../logs2/logs"

In [81]:
from tbparse import SummaryReader
reader = SummaryReader(log_dir, pivot=True, extra_columns={'dir_name'})
df = reader.hparams

hparams_dict = df.set_index('dir_name').T.to_dict()
print(hparams_dict)  # dictionary of hparams

{'sketch_transformer_example0/20251109_220241': {'d_model': [256.0, 256.0, 256.0], 'lr': [0.0001, 0.0001, 0.0001], 'max_len': [200.0, 200.0, 200.0], 'model': ['SketchTransformer', 'SketchTransformer', 'SketchTransformer'], 'n_layers': [6.0, 6.0, 6.0], 'nhead': [8.0, 8.0, 8.0], 'tokenizer': ['AbsolutePenPositionTokenizer', 'AbsolutePenPositionTokenizer', 'AbsolutePenPositionTokenizer'], 'tokenizer_bins': [32.0, 32.0, 32.0], 'use_padding_mask': [0.0, 0.0, 0.0]}, 'sketch_transformer_example0/20251109_221602': {'d_model': 384.0, 'lr': 0.0001, 'max_len': 200.0, 'model': 'SketchTransformer', 'n_layers': 6.0, 'nhead': 8.0, 'tokenizer': 'AbsolutePenPositionTokenizer', 'tokenizer_bins': 32.0, 'use_padding_mask': 0.0}, 'sketch_transformer_example0/20251109_223152': {'d_model': 256.0, 'lr': 0.0001, 'max_len': 200.0, 'model': 'SketchTransformer', 'n_layers': 8.0, 'nhead': 8.0, 'tokenizer': 'AbsolutePenPositionTokenizer', 'tokenizer_bins': 32.0, 'use_padding_mask': 0.0}}


In [74]:
from tensorboard.backend.event_processing import event_accumulator
import os

def get_latest_step(log_dir: str):
    """
    Recursively search log_dir for TensorBoard event files, pick the most recently
    modified one, and return the largest recorded step among all scalar tags.
    """
    # Collect event files recursively
    event_files = []
    for root, _, files in os.walk(log_dir):
        for f in files:
            if f.startswith("events.out.tfevents"):
                event_files.append(os.path.join(root, f))

    if not event_files:
        raise FileNotFoundError("No TensorBoard event files found in " + log_dir)

    # Sort by modification time (latest last)
    event_files.sort(key=lambda p: os.path.getmtime(p))
    latest_event_file = event_files[-1]

    # Load the event file
    ea = event_accumulator.EventAccumulator(
        latest_event_file,
        size_guidance={"scalars": 0}  # load all scalars
    )
    ea.Reload()

    # Gather all scalar tags and steps
    all_steps = []
    for tag in ea.Tags().get("scalars", []):
        events = ea.Scalars(tag)
        if events:
            all_steps.append(events[-1].step)

    if not all_steps:
        raise ValueError("No scalar steps found in " + latest_event_file)

    return max(all_steps)

# Example
if __name__ == "__main__":
    latest_step = get_latest_step("../logs")
    print("Latest step:", latest_step)

ValueError: No scalar steps found in ../logs\sketch_transformer_experiment_0\1762737355.1614318\events.out.tfevents.1762737355.DESKTOP-6UC3P7J.28764.9

In [90]:
from tensorboard.backend.event_processing import event_accumulator
import os

def get_latest_step(log_dir: str):
    # Find the latest TensorBoard event file
    event_files = [f for f in os.listdir(log_dir) if f.startswith("events.out.tfevents")]
    if not event_files:
        raise FileNotFoundError("No TensorBoard event files found in " + log_dir)
    
    # Sort by modification time (latest last)
    event_files = sorted(event_files, key=lambda f: os.path.getmtime(os.path.join(log_dir, f)))
    latest_event_file = os.path.join(log_dir, event_files[-1])

    # Load the event file
    ea = event_accumulator.EventAccumulator(latest_event_file)
    ea.Reload()

    # Gather all scalar tags and steps
    all_steps = []
    for tag in ea.Tags()['scalars']:
        events = ea.Scalars(tag)
        if events:
            all_steps.append(events[-1].step)

    if not all_steps:
        raise ValueError("No scalar steps found in " + log_dir)
    
    return max(all_steps)

# Example
latest_step = get_latest_step("../logs2/logs/sketch_transformer_example0/20251109_220241")
print("Latest step:", latest_step)

Latest step: 15


In [93]:
from tensorboard.backend.event_processing import event_accumulator
import os
from typing import List, Dict, Any, Iterable, Tuple, Union

def _event_files_under(path: str) -> List[str]:
    if os.path.isfile(path):
        return [path] if os.path.basename(path).startswith("events.out.tfevents") else []
    files = []
    for root, _, fs in os.walk(path):
        for f in fs:
            if f.startswith("events.out.tfevents"):
                files.append(os.path.join(root, f))
    return sorted(files, key=os.path.getmtime)

def load_scalar_records(path: str) -> List[Dict[str, Any]]:
    """
    Return a LONG list of records: [{'file', 'tag', 'step', 'value', 'wall_time'} ...]
    Each record is a scalar value logged at a specific step.
    """
    records: List[Dict[str, Any]] = []
    files = _event_files_under(path)
    if not files:
        raise FileNotFoundError(f"No TensorBoard event files found under: {path}")

    for evfile in files:
        ea = event_accumulator.EventAccumulator(evfile)
        ea.Reload()
        for tag in ea.Tags().get('scalars', []):
            for ev in ea.Scalars(tag):
                records.append({
                    "file": evfile,
                    "tag": tag,
                    "step": ev.step,
                    "value": ev.value,
                    "wall_time": ev.wall_time,
                })
    if not records:
        raise ValueError(f"No scalar values found in: {path}")
    return records

def group_scalars_by_step(records: Iterable[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Collapse long records into one dict per step with all tags at that step.
    If multiple files/runs have the same step+tag, the last (newest) by wall_time wins.
    Returns a list sorted by step: [{'step', 'wall_time', 'file', '<tag1>': v1, ...}, ...]
    """
    by_step: Dict[int, Dict[str, Any]] = {}
    for r in records:
        s = r["step"]
        bucket = by_step.setdefault(s, {"step": s, "wall_time": r["wall_time"], "file": r["file"]})
        # Keep the newest wall_time/file if we see same step again
        if r["wall_time"] >= bucket["wall_time"]:
            bucket["wall_time"] = r["wall_time"]
            bucket["file"] = r["file"]
        bucket[r["tag"]] = r["value"]
    return sorted(by_step.values(), key=lambda d: d["step"])

def latest_step_info(path: str) -> Tuple[int, Dict[str, Any]]:
    """
    Convenience: return (latest_step, merged_row_for_that_step)
    """
    records = load_scalar_records(path)
    by_step = group_scalars_by_step(records)
    row = by_step[-1]
    return row["step"], row

# ---------- Usage ----------
# 1) Get every scalar point (long format):
records = load_scalar_records("../logs2/logs/sketch_transformer_example0/20251109_220241")
print(f"Loaded {len(records)} scalar points. Example:", records[:3])

# 2) Group by step (wide format: one row per step, columns are tags):
by_step = group_scalars_by_step(records)
print("First 3 steps (wide rows):")
for row in by_step[:15]:
    print(row)

# 3) Latest step with all its tags:
step, row = latest_step_info("../logs2/logs/sketch_transformer_example0/20251109_220241")
print(f"Latest step: {step}")
print("All scalars at latest step:", {k: v for k, v in row.items() if k not in ('step','wall_time','file')})

Loaded 64 scalar points. Example: [{'file': '../logs2/logs/sketch_transformer_example0/20251109_220241\\events.out.tfevents.1762743761.DESKTOP-99OM8G7.10288.0', 'tag': 'Loss/Train', 'step': 0, 'value': 2.116799831390381, 'wall_time': 1762743798.6510417}, {'file': '../logs2/logs/sketch_transformer_example0/20251109_220241\\events.out.tfevents.1762743761.DESKTOP-99OM8G7.10288.0', 'tag': 'Loss/Val', 'step': 0, 'value': 1.815959095954895, 'wall_time': 1762743799.9395761}, {'file': '../logs2/logs/sketch_transformer_example0/20251109_220241\\events.out.tfevents.1762743761.DESKTOP-99OM8G7.10288.0', 'tag': 'Accuracy/ValNextToken', 'step': 0, 'value': 0.13910172879695892, 'wall_time': 1762743799.9397447}]
First 3 steps (wide rows):
{'step': 0, 'wall_time': 1762743799.9504473, 'file': '../logs2/logs/sketch_transformer_example0/20251109_220241\\events.out.tfevents.1762743761.DESKTOP-99OM8G7.10288.0', 'Loss/Train': 2.116799831390381, 'Loss/Val': 1.815959095954895, 'Accuracy/ValNextToken': 0.139101

In [94]:
import os
from typing import List, Dict, Any
from tensorboard.backend.event_processing import event_accumulator

# These protos are part of TensorBoard's HParams plugin
from tensorboard.plugins.hparams import api_pb2


def _event_files_under(path: str) -> List[str]:
    if os.path.isfile(path):
        return [path] if os.path.basename(path).startswith("events.out.tfevents") else []
    files = []
    for root, _, fs in os.walk(path):
        for f in fs:
            if f.startswith("events.out.tfevents"):
                files.append(os.path.join(root, f))
    return sorted(files, key=os.path.getmtime)


def _hp_value_to_python(pb_val: api_pb2.HParamValue) -> Any:
    # HParamValue oneof typically is 'string_value', 'number_value', or 'bool_value'
    kind = pb_val.WhichOneof("kind")
    if kind == "string_value":
        return pb_val.string_value
    if kind == "number_value":
        # Cast to float or int if it is effectively integral
        v = pb_val.number_value
        return int(v) if float(v).is_integer() else float(v)
    if kind == "bool_value":
        return pb_val.bool_value
    return None  # unknown kind


def _try_parse_session_start(tensor_proto) -> api_pb2.SessionStartInfo | None:
    """
    Attempt to parse a tensor's first string_val as a SessionStartInfo.
    Returns the parsed proto or None on failure.
    """
    try:
        if not tensor_proto.string_val:
            return None
        raw = tensor_proto.string_val[0]
        info = api_pb2.SessionStartInfo()
        info.ParseFromString(raw)
        # Heuristic: must have at least session_id or some hparams to be valid
        if info.session_id or len(info.hparams) > 0:
            return info
    except Exception:
        return None
    return info


def load_hparams_sessions(path: str) -> List[Dict[str, Any]]:
    """
    Extract HParams session starts from TensorBoard event files under `path`.

    Returns a list like:
    [
      {
        "file": ".../events.out.tfevents....",
        "wall_time": 1731192000.123,
        "session_id": "abc123",
        "group_name": "exp_group",
        "hparams": {"lr": 0.001, "batch_size": 64, "model": "transformer", ...}
      },
      ...
    ]
    """
    files = _event_files_under(path)
    if not files:
        raise FileNotFoundError(f"No TensorBoard event files found under: {path}")

    sessions: List[Dict[str, Any]] = []

    # Size guidance: ensure tensors are loaded
    size_guidance = {
        event_accumulator.TENSORS: 0,
        event_accumulator.SCALARS: 0,
        event_accumulator.HISTOGRAMS: 0,
        event_accumulator.IMAGES: 0,
        event_accumulator.COMPRESSED_HISTOGRAMS: 0,
    }

    for evfile in files:
        ea = event_accumulator.EventAccumulator(evfile, size_guidance=size_guidance)
        ea.Reload()

        tensor_tags = ea.Tags().get("tensors", []) or []
        # Prefer obvious names first, but we'll try all tags
        preferred_order = ["hparams", "hp/hparams", "hparams/session_start_info"]
        ordered_tags = sorted(tensor_tags, key=lambda t: (t not in preferred_order, t))

        for tag in ordered_tags:
            for t in ea.Tensors(tag):
                info = _try_parse_session_start(t.tensor_proto)
                if info is None:
                    continue

                # Convert map<string, HParamValue> to python dict
                hp_dict = {k: _hp_value_to_python(v) for k, v in info.hparams.items()}

                sessions.append({
                    "file": evfile,
                    "wall_time": t.wall_time,
                    "session_id": info.session_id,     # may be empty if not set
                    "group_name": info.group_name,     # may be empty if not set
                    "hparams": hp_dict,
                })

    if not sessions:
        raise ValueError(
            "No HParams (SessionStartInfo) found. "
            "Make sure you logged hparams via TF HParams plugin or torch.add_hparams()."
        )

    # Sort sessions by time, newest last
    sessions.sort(key=lambda s: s["wall_time"])
    return sessions


# ---------- Usage ----------
# 1) Load all HParams sessions under a run directory or a single event file:
hparam_sessions = load_hparams_sessions("../logs2/logs/sketch_transformer_example0/20251109_220241")
print(f"Found {len(hparam_sessions)} HParams session(s). Example:")
print(hparam_sessions[-1])  # newest

# 2) If you just want the "latest" hparams dict:
latest_hparams = hparam_sessions[-1]["hparams"]
print("Latest HParams:", latest_hparams)

# 3) If you want to merge multiple sessions (last value wins) into one dict:
merged = {}
for s in hparam_sessions:
    merged.update(s["hparams"])
print("Merged HParams across sessions:", merged)

AttributeError: module 'tensorboard.plugins.hparams.api_pb2' has no attribute 'HParamValue'

In [ ]:
import os
print(os.getcwd())          # shows your current working directory
print(os.listdir("."))  # see what subfolders exist

c:\Users\sebas\Generative-SVG\experiments


FileNotFoundError: [WinError 3] The system cannot find the path specified: './logs2'